In [201]:
import time
import os
import sys
import re
import pandas
import numpy
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from PIL import Image
home_dir = os.environ.get("HOME")
deltas = pandas.read_excel(f"{home_dir}/Downloads/Deltas.xlsx")

In [171]:

dates_to_screenshot = [pandas.Timestamp(val) for val in deltas.Date.values]
GECKODRIVER_PATH = '/usr/local/bin/geckodriver'
profiles_folder = f'{home_dir}/Library/Caches/Firefox/Profiles/'
default_profile = os.listdir(profiles_folder)[-1]
profile_path = f'{profiles_folder}/{default_profile}'
screenshot_path = f"{home_dir}/Downloads/capitaloneshopping_rewards_evidence"
if not os.path.exists(screenshot_path): os.mkdir(screenshot_path)
options = Options()
options.add_argument("-profile")
options.add_argument(profile_path)
service = Service(GECKODRIVER_PATH)
driver = webdriver.Firefox(service=service, options=options)
instacart_url = "https://www.instacart.com/store/account/orders"
driver.get(instacart_url)
input("""Hit enter after you've logged in and loaded all the orders on the page and scrolled to the top""")
if driver.current_url != instacart_url:
    driver.get(instacart_url)
    time.sleep(2)

Hit enter after you've logged in and loaded all the orders on the page and scrolled to the top


In [172]:
driver.execute_script("document.body.style.transform = 'scale(0.3)';")
driver.execute_script("document.body.style.transformOrigin = '0 0';")
order_details = driver.find_elements(By.LINK_TEXT, 'View order detail')

In [173]:
for order_detail in order_details[36:]:
    action = ActionChains(driver)
    driver.execute_script("arguments[0].scrollIntoView(true);", order_detail)
    action.key_down(Keys.COMMAND).click(order_detail).key_up(Keys.COMMAND).perform()
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(3)
    driver.execute_script("document.body.style.transform = 'scale(0.3)';")
    driver.execute_script("document.body.style.transformOrigin = '0 0';")
    time.sleep(3)
    if not 'Order canceled' in driver.page_source:
        receipt_link = driver.find_element(By.PARTIAL_LINK_TEXT,'Receipt')
        receipt_link.click()
        time.sleep(2)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(3)
        driver.execute_script("document.body.style.transform = 'scale(0.3)';")
        driver.execute_script("document.body.style.transformOrigin = '0 0';")
        time.sleep(3)
        schedule_text = driver.find_element(By.CLASS_NAME,'DriverDeliverySchedule').text
        order_date = re.search('(?<=placed on ).+(?= and)',schedule_text).group(0)
        if pandas.Timestamp(order_date) in dates_to_screenshot:
            amount = driver.find_element(By.CLASS_NAME,'amount').text
            driver.save_full_page_screenshot(f"{screenshot_path}/{order_date} {amount}.png")
    driver.close()
    driver.switch_to.window(driver.window_handles[-1])
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
driver.quit()

In [219]:
files = sorted(os.listdir(screenshot_path), key=lambda x: os.stat(os.path.join(screenshot_path, x)).st_birthtime)
files = [x for x in files if not x.startswith('.')]

In [ ]:
crop_box = (0, 0, 800, 1100)
for filename in files:
    if filename.startswith('.'):
        continue
    image = Image.open(f'{screenshot_path}/{filename}')
    cropped_image = image.crop(crop_box)
    cropped_image.save(f'{screenshot_path}/{filename}')

In [220]:
trip_numbers = [t.split(", ") for t in deltas['Trip Number(s)'].values]
trip_numbers = [item for sublist in trip_numbers for item in sublist]

In [221]:
for filename, trip_number in zip(files, trip_numbers):
    os.rename(f'{screenshot_path}/{filename}', f"{screenshot_path}/{filename.split(',')[0]}, {trip_number}.png")